In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../zkstats/core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

# where(element > 30)
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.mean(gt30_dummy_data_tensor)



In [5]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.new_X = nn.Parameter(data = gt30_dummy_data_tensor,requires_grad = False )

    def forward(self,X):
        # where part
        # Many of these implementations are weird, but make it satisfy zkp of ezkl.
        len_ratio =  self.new_X.size()[1]/X.size()[1]
        X_where = torch.zeros(len_ratio*X.size()[1]).reshape(1,-1,1)
        X_where[0]=self.new_X[0]
        # constraint that new_X is indeed X where element is greater than 30
        new_X_cons = torch.sum((torch.abs(X[X>30].reshape(1,-1,1)-X_where)<=torch.abs(0.01*X_where)).double())==X_where.size()[1]

        # can't put new_X_cons directly into return
        if new_X_cons:
            # value from mean calculation
            value = torch.abs(torch.sum(X_where)-X_where.size()[1]*(self.w))<=torch.abs(0.01*X_where.size()[1]*self.w)
        else:
            # return false aka 0
            value = X_where.size()[1]<0

        return (value, self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_58237/4035532840.py:19: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if new_X_cons:
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:ConstantOfShape, node name: /ConstantOfShape): input typestr: T1, has unsupported type: tensor(float) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/serialization/export.cp

In [6]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
print("new tensor: ", gt30_data_tensor.size())
# gt30_data_tensor = torch.reshape(gt30_data_tensor_raw, (1, int(gt30_data_tensor_raw.size()[0]), 1))
theory_output = torch.mean(gt30_data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.new_X = nn.Parameter(data = gt30_data_tensor,requires_grad = False )
    def forward(self,X):
        # where part
        # Many of these implementations are weird, but make it satisfy zkp of ezkl.
        len_ratio =  self.new_X.size()[1]/X.size()[1]
        X_where = torch.zeros(len_ratio*X.size()[1]).reshape(1,-1,1)
        X_where[0]=self.new_X[0]
        # constraint that new_X is indeed X where element is greater than 30
        new_X_cons = torch.sum((torch.abs(X[X>30].reshape(1,-1,1)-X_where)<=torch.abs(0.01*X_where)).double())==X_where.size()[1]

        # can't put new_X_cons directly into return
        if new_X_cons:
            # value from mean calculation
            value = torch.abs(torch.sum(X_where)-X_where.size()[1]*(self.w))<=torch.abs(0.01*X_where.size()[1]*self.w)
        else:
            # return false aka 0
            value = X_where.size()[1]<0

        return (value, self.w)
    

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

new tensor:  torch.Size([1, 272, 1])
Theory_output:  tensor(52.3676)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_58237/605661225.py:23: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if new_X_cons:


In [7]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.1477460861206055 seconds
Theory output:  tensor(52.3676)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 52.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[17970410297053518904, 9046703063145816218, 2851759239208196922, 164045840560226117], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [1460628720732602093, 15005283833837523956, 127539993950163949, 306168462079750959]]], 'proof': '1289303ff75cc3a36210dc5e377af615ad526dfbfc4387a5a8f7a44ee62719ce20e1e2fc3ffb67c5971c242f10f2671dc5a17ec7d0ee05c3311030ec726089521166dec04b07cfb114a6b7ea458944358fcfad10411f0a83a3e94c3b1e72dea620e1ccb7c0aa14cc54f5a1f5778ecaecffaa1e517dec3323afa66ca3b6be2e512232240181e6e51423118aae1e7663755002c14da7c0504beed4f5f7dd6a07591e97f8d2e537449c97778d00388bdaf276e340095f159fce486bc863254327bc1b73b288190efbf9163bddfce9563ab6667b6bca5ce0e98fe065dd6a85dab79906a71fd2b1be61db96aa1c4397ea48431661dee3831acab659cc07f52d9f85e621a26b64dcfd9b1c23ef2569ffb55f3b84dcb6d2d3a6b7d579a0cf2b05e53f442ff33a97e06d698ec2c50acb148138b8cb41fd52bc23efb4182e0ac5712195030a730bb35847c220ec8c78b0f320bbae00a136fe0753fd88d1ea1c067f3da98806

In [8]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[17970410297053518904, 9046703063145816218, 2851759239208196922, 164045840560226117], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [1460628720732602093, 15005283833837523956, 127539993950163949, 306168462079750959]]]
proof boolean:  1.0
proof result 1 : 52.0
verified
